# 文本挖掘（一）
*Copyright shiboshen19s@ict.ac.cn*
## 通过本节笔记你将学到：


1.   词的表达：独热编码与hash tricking/分布式表达LSI PLSI LDA ida2vec/神经网络模型word2vec等
2.   句的表达

本节笔记代码可能不是很多。代码和演示将在下一节笔记中给出。

另求NLP大腿补充...



## 1.独热编码与hash tricking

词的独热编码的处理过程是：收集所有文档进行分词，建立大词汇表一共包含m个单词，然后建立字典{word:idx}。这个idx可以是按照次序来的1，2，3...则每个单词对应m维向量，第idx个entry=1, 其他entry=0。和bitmap一个道理。

缺点： 当词汇表庞大时词向量过于稀疏，引起“维数灾难”；而且词向量之间彼此正交，失去了上下文语义信息（因为我们认为在word space中，表意相近的单词距离度量较小。距离度量=欧几里得距离/余弦相似度/pearson系数等）

当idx通过hash得到时，就变成了一种hash tricking，即它占的“1”坑位通过hash得到。同时词向量的维数可以比词汇表大一些，根据计算公式：$idx=hash(word)%output_dim$,这个output_dim可以自己定义，但是小于词汇表长度的话可能会引起hash碰撞。hash函数的性质保证了相同的单词的坑位是一致的。

实验发现当output_dim比词汇表稍大一点时，使用hash tricking比one-hot编码要更理想一些，不会因多次实验而出现较大的性能波动而且和最优值接近。

[hash tricking](http://www.willmcginnis.com/2016/01/16/even-further-beyond-one-hot-hashing/)

In [0]:
import numpy as np

samples = ['The cat sat on the mat.', 'The dog ate my homework.']
dimensionality = 1000
max_length = 10

results = np.zeros((len(samples), max_length, dimensionality))
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        # the word is given a hash with a random integer
        index = abs(hash(word)) % dimensionality
        results[i, j, index] = 1.0

np.where(results>0)

#通过输出结果对照发现，两个句子中的"the"，索引是(0,0)和(1,0)，占的坑位都是768号。

(array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]),
 array([0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4]),
 array([768, 306,   6,  58, 486, 668, 768,  44, 492, 536,  10]))

##2-1.LSI模型（分布式横向建模，主题模型）

当引入分布式表达时，就有两种研究词语之间的关系：横向的叫组合（共现），纵向的叫替换，即同一个词的上下文应该相似。LSI PLSI LDA都是横向建模，基于上下文的建模如word2vec都是纵向建模。分布式表达通过训练，将每个词都映射到一个较短的词向量上来。所有的这些词向量就构成了向量空间，进而可以用普通的统计学的方法来研究词与词之间的关系。

LSI模型是主题模型的一种。首先构建一个单词-文档矩阵C，即$C_{ij}$是第i个单词在第j篇文档里的tf-idf值。在这里引入一个主题的概念：每篇文档（C的列向量）所在空间的基代表一系列主题，每个文档列向量是这些“主题基向量”的组合。

通过SVD分解我们得到C的k秩近似C'。SVD分解去除了噪声、相似的“主题基向量”也得到了整合。新的k维空间中每个新基代表一系列截然不同的主题。当用C'表示新的单词-文档矩阵时，我们就能用它做一系列事情。比如文档相似度度量：给一个新的文档向量，可以和C'中的每列依次做距离度量看看离哪个最近。当然行向量可以拿来做单词相似度度量。或者查询词最可能在哪篇文档中（？）。

然鹅这个可解释性很差。主题这个概念没有一个数学表达，更像是一种信念"belief",我们相信词共现、句子组合成段落这些是受到主题掌控的但是没有具体证据。 PLSI, LDA通过从概率模型的角度引入主题隐变量建模，解决了这个问题。 ida2vec从神经网络建模的角度解决这个问题。

LSI实现很简单。



https://www.zhihu.com/question/26751755

https://www.datacamp.com/community/tutorials/lda2vec-topic-model



In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
documents = ["doc1.txt", "doc2.txt", "doc3.txt"] #把每个字符串替换为真正的文档才能运行

# raw documents to tf-idf matrix: 
vectorizer = TfidfVectorizer(stop_words='english', 
                             use_idf=True, 
                             smooth_idf=True)
# SVD to reduce dimensionality: 
svd_model = TruncatedSVD(n_components=100,       
                         algorithm='randomized',
                         n_iter=10)
# pipeline of tf-idf + SVD, fit to and applied to documents:
svd_transformer = Pipeline([('tfidf', vectorizer), 
                            ('svd', svd_model)])
svd_matrix = svd_transformer.fit_transform(documents)

# svd_matrix can later be used to compare documents, compare words, or compare queries with documents

## 2-2. PLSI模型（分布式横向建模，主题模型）

主题在这里从信念变成了隐变量，通过建立贝叶斯网络对词语-文档的出现进行概率建模。

d-文档  z-主题 w-单词

根据贝叶斯网络：$p(d_{m},z_{k},w_{n}) = p(d_{m})p(z_{k}|d_{m})p(w_{n}|z_{k})$

建模文档与单词的联合分布：$p(d_{m},w_{n})=p(d_{m})\sum_{k}p(z_{k}|d_{m})p(w_{n}|z_{k})=\sum_{k}p(z_{k})p(d_{m}|z_{k})p(d_{m}|z_{k})$

前一种由于在这个式子里面单词和文档的生成并不相似， 所以是非对称形式。文档的概率作为一种先验，首先根据文档生成主题分布，之后根据主题生成单词分布。后一种是对称形式，即条件概率的条件均为z：主题的分布作为先验，之后根据主题派生出文档和单词的分布。

概率模型就是要估计参数的。PLSI估计的就是$p(d_{m},w_{n})$: 比如生成一篇文档时，可以粗略地认为$p(d_{m},W)=\sum_{n}logp(d_{m},w_{n})$。根据以上非对称形式，需要估计两个参数$p(z_{k}|d_{m})and p(w_{n}|z_{k})$。最大似然估计+EM算法估计。

EM算法的结果很多都是最大似然下的频率估计结果。这是正确的。参考书P166页的结果。它本质上就是频率估计的结果：分母是归一化常数；分子中$n(d_{m},w_{n})就是词w_{n}和文档d_{m}共现的次数，要乘上概率p(z_{k}|...)类似于期望，最后对文档求和即可$

缺点：该模型不能产生新文档，因为根据非对称的公式$p(d_{m})$对新文档是未知的，这样参数不能估计就什么都干不了。而且随着文档数量的增加,$P(z|d)$的参数也会随着线性增加，这就导致无论有多少训练数据，都容易导致模型的过拟合问题。这两点成为了限制 PLSA 模型被更加广泛使用的两大缺陷。

##2-3. LDA模型（分布式横向建模，主题模型）


---
**首先研究一下“分布的分布”**。比如投硬币，我们大家都知道结果服从二项分布b(n,p)。好像大家一般默认p=0.5，但是如果我们引入p的分布，比如均匀分布p~Uniform(0,1),就是一种“分布的分布”。

再看看狄利克雷分布。先从beta分布开始。它们都是“分布的分布”。beta分布是假设变量服从二项分布时对那个p引入的分布。它特别好的一点在于使用贝叶斯公式时在学习过程中后验分布和先验分布形式一致，只是参数不断得到修正。这样先验分布和后验分布形式相同时，我们称他们为共轭分布（only a term,不用管它）。

$P(p|x)=\frac{P(x|p)P(p)}{P(x)},这里P(x|p)就是二项分布公式，P(p)就是beta分布，下面就是归一化常数$ 最后的结果P(p|x)和P(p)都是beta分布，只不过对初始参数进行了修正。这就是一个bayes学习过程。

狄利克雷分布只不过把二项分布推广成多项式分布，即每次取值集合扩大了。



[一个很好的解释](https://www.zhihu.com/question/26751755)



---

**那么狄利克雷分布和主题有什么关系呢？**在PLSI模型中我们对文档进行抽样，然后根据该文档抽样主题，再根据该主题抽样一个单词。它只是引入了主题隐变量，并没有对它建模。在LDA中每一个文档都被看成跟有限个给定主题中的每一个存在着或多或少的关联性，而这种关联性则是用主题上的概率分布来刻画的。因为主题是符合多项式分布的，而多项式分布中每一个参数又未知，所以对于主题分布的参数$\theta$就使用了狄利克雷先验。从LDA过程中也可以看出来。（书P168，略）

此外，LDA 模型还对一个主题在所有单词上的概率分布也赋予了一个稀疏形式的狄利克雷先验，它的直观解释也是类似的:在一个单独的主题中，多数情况是少部分(跟这个主题高度相关的)词出现的频率会很高，而其他的词出现的频率则明显较低。这样两种先验使得 LDA 模型能够比 PLSA 更好地刻画文档-话题-单词这三者的关系。通常而言，LDA 比 pLSA 效果更好，因为它可以轻而易举地泛化到新文档中去。在 pLSA 中，文档概率是数据集中的一个固定点。如果没有看到那个文件，我们就没有那个数据点。然而，在 LDA 中，数据集作为训练数据用于文档-主题分布的狄利克雷分布。即使没有看到某个文件，我们可以很容易地从狄利克雷分布中抽样得来，并继续接下来的操作。


通过使用 LDA，我们可以从文档语料库中提取人类可解释的主题，其中每个主题都以与之关联度最高的词语作为特征。例如，主题i“宅舞”可以用诸如「二次元，舞蹈，萌妹子，可爱，小姐姐」等术语来表示。这个的做法是：既然我们知道了主题分布等参数，就能遍历该文本的词汇表寻找哪些词汇使得特定主题下生成该文本的概率最大。比如对于主题i有n个词汇使得生成文档的概率最大，则表示主题i的词就是它。执行时要对所有文档遍历一遍。此外，在给定一个新文档的条件下，我们可以获得表示其主题混合的向量，例如，5％ 的主题 1，70％ 的主题 2，10％的主题 3 等。通常来说，这些向量对下游应用非常有用。主题向量在ida2vec里有用到。gensim包可以调用LDA api。

[补充阅读](https://www.jiqizhixin.com/articles/2019-06-12-8)


[在中文文本上使用LDA编程的详细例子](https://blog.csdn.net/Yellow_python/article/details/83097994)


In [0]:
from gensim import corpora, models
import jieba.posseg as jp, jieba
# 文本集
texts = [
    '美国教练坦言，没输给中国女排，是输给了郎平' * 99,
    '美国无缘四强，听听主教练的评价' * 99,
    '中国女排晋级世锦赛四强，全面解析主教练郎平的执教艺术' * 99,
    '为什么越来越多的人买MPV，而放弃SUV？跑一趟长途就知道了' * 99,
    '跑了长途才知道，SUV和轿车之间的差距' * 99,
    '家用的轿车买什么好' * 99]
# 分词过滤条件
jieba.add_word('四强', 9, 'n')
flags = ('n', 'nr', 'ns', 'nt', 'eng', 'v', 'd')  # 词性
stopwords = ('没', '就', '知道', '是', '才', '听听', '坦言', '全面', '越来越', '评价', '放弃', '人')  # 停词
# 分词
words_ls = []
for text in texts:
    words = [w.word for w in jp.cut(text) if w.flag in flags and w.word not in stopwords]
    words_ls.append(words)
# 构造词典
dictionary = corpora.Dictionary(words_ls)
# 基于词典，使【词】→【稀疏向量】，并将向量放入列表，形成【稀疏向量集】
corpus = [dictionary.doc2bow(words) for words in words_ls]
# lda模型，num_topics设置主题的个数
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=2)
# 打印所有主题，每个主题显示5个词
for topic in lda.print_topics(num_words=5):
    print(topic)
# 主题推断
print(lda.inference(corpus))


##3-1. NPLM和word2vec（分布式建模，纵向）

使用词的上下文训练词向量的时代开启了。

NPLM即用DNN训练词向量，也分为CBOW和skip-gram两种模式。


1.   对CBOW来说，输入是中心词上下文window_size*2大小的词向量，输出层有词汇表大小的神经元，使用softmax计算每个神经元的输出，即对应中心词的产生概率。之后用argmax找最大的概率的索引对应的单词作为中心词预测输出。
2.   skip-gram只不过是反过来，输入是一个词向量，输出层有词汇表大小的神经元，softamx计算前2*window_size大小个单词作为上下文预测输出。

很明显，对于输出层有词汇表大小的向量做softmax非常耗时。

谷歌的word2vec并不是用DNN架构，而是采用了huffman tree引入了结构化softmax或者negative sampling两种不同的技术路线分别解决softmax计算耗时等问题。使用huffman树有什么好处呢？首先，由于是二叉树，之前计算量为𝑉,现在变成了𝑙𝑜𝑔2𝑉。第二，由于huffman树是高频的词靠近树根，这样高频词需要更少的时间会被找到，这符合我们的贪心优化思想。而且它的输入不是一个词向量矩阵而是窗口中所有向量的平均，只有一个一维输入向量。注意这里的huffman树：约定左子树编码为1，右子树编码为0，同时约定左子树的权重不小于右子树的权重。
[参考这个系列的博客！](https://www.cnblogs.com/pinard/p/7243513.html)

这里总结下基于Hierarchical Softmax的CBOW模型算法流程，梯度迭代使用了随机梯度上升法：

　　　　输入：基于CBOW的语料训练样本，词向量的维度大小𝑀，CBOW的上下文大小2𝑐,学习率𝜂

　　　　输出：霍夫曼树的内部节点模型参数𝜃，所有的词向量𝑤

　　　　1. 基于语料训练样本建立词汇表的霍夫曼树。频次越高的词越靠近根节点。

　　　　2. 随机初始化所有的模型参数𝜃，所有的词向量𝑤

　　　　3. 进行梯度上升迭代过程。注意这里是SGD，即每次只拿一个样本进行参数更新。细节参考博客。
skip-gram类似。

负采样是一个很经典的算法，可以拎出来用到其他地方。

现在我们来看看如何进行负采样，得到neg个负例。word2vec采样的方法并不复杂，如果词汇表的大小为𝑉,那么我们就将一段长度为1的线段分成𝑉份，每份对应词汇表中的一个词。当然每个词对应的线段长度是不一样的，高频词对应的线段长，低频词对应的线段短。每个词𝑤的线段长度由下式决定：
$𝑙𝑒𝑛(𝑤)=\frac{𝑐𝑜𝑢𝑛𝑡(𝑤)}{\sum_{𝑢∈𝑣𝑜𝑐𝑎𝑏}𝑐𝑜𝑢𝑛𝑡(𝑢)}$

在word2vec中，分子和分母都取了3/4次幂如下：
$𝑙𝑒𝑛(𝑤)=\frac{𝑐𝑜𝑢𝑛𝑡(𝑤)^{3/4}}{\sum_{𝑢∈𝑣𝑜𝑐𝑎𝑏}𝑐𝑜𝑢𝑛𝑡(𝑢)^{3/4}}$

在采样前，我们将这段长度为1的线段划分成𝑀等份，这里𝑀>>𝑉，这样可以保证每个词对应的线段都会划分成对应的小块。而M份中的每一份都会落在某一个词对应的线段上。在采样的时候，我们只需要从𝑀个位置中采样出𝑛𝑒𝑔个位置就行，此时采样到的每一个位置对应到的线段所属的词就是我们的负例词。

这里总结下基于Negative Sampling的CBOW模型算法流程，梯度迭代使用了随机梯度上升法：

输入：基于CBOW的语料训练样本，词向量的维度大小𝑀𝑐𝑜𝑢𝑛𝑡，CBOW的上下文大小2𝑐,学习率𝜂, 负采样的个数neg

输出：词汇表每个词对应的模型参数𝜃，所有的词向量𝑥𝑤

　　　　1. 随机初始化所有的模型参数𝜃，所有的词向量𝑤

　　　　2. 对于每个训练样本(𝑐𝑜𝑛𝑡𝑒𝑥𝑡(𝑤0),𝑤0),负采样出neg个负例中心词𝑤𝑖,𝑖=1,2,...𝑛𝑒𝑔

　　　　3. 进行梯度上升迭代过程。

skip-gram类似


---

训练word2vec可以有gensim包。另外也有一种ngram2vec的开源方法在github上。如何使用word2vec训练好的词向量？把它作为keras embedding层的权矩阵并且标注为不可训练即可。输入是文档的词索引表示矩阵。



##3-2. ida2vec
https://github.com/cemoody/lda2vec  主题变成了一个可训练的向量

##4-1. 句子向量

如果我们得到了n维词向量，而一个句子有若干个词，总不可能把所有词向量依次拼起来得到句子向量吧！

别急，先看看一些传统方法。

非分布式的表达包括word set 和word bag模型。word set模型把句向量表达成一个文档词汇表大小的0-1向量。若在该句子中出现某单词则在它的索引下面标1，否则是0。word bag模型把0-1扩展成出现次数。此基础上还可以把次数替换成tf-idf数值。tfidf一句话就是某一特定文档里的高频词语，以及该词语出现在其他文档中的频率低，tfidf数值就高。

$tfidf_{i,j}=tf_{i,j}*idf_{i}=\frac{n_{i,j}}{\sum_{k}n_{i,k}}*log\frac{|D|}{df_{i}}, df_{i}指包含单词i的文档数$  

分布式表达书上介绍的方法除了LDA都是深度学习方法。把句子当成文档处理，LDA就能把句子表示成稠密向量。其他dl方法暂时先pass，以后再说。